# **Basic MoSDeF Workflow**

## Introduction for Lyon lab members
There are two ways to use this Jupyter Notebook:

1. Installing Anaconda (anaconda.org) or Miniconda in your computer and running locally this notebook.

2. Accesing this notebook through the <a href="https://github.com/lyon-group/Intro_to_MoSDeF">Lyon Group GitHub repository</a> and opening it either in Google Collab or GitHub workspaces (Google Collab is recommended if using Chapman email).


## Important considerations
There are two types of output in these Colab notebooks that can be a little trick

1. If the output is very long, for example from the mamba command in the second cell, scrolling past the output can feel onerous. In this case, scrolling up and down in the narrow grey area between the sidebar menu and the cells can help you navigate.

2. If the output is a visualization of a molecule or simulation configuration, scrolling up or down will zoom in or out if the cursor is over the visualization. In these cases, take some care to scroll outside of the visualization.

3. To run a cell, either click the run button (right facing triangle) or hit `shift + enter`

In [1]:
import warnings
#warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Import Libraries to be used
import os
import mbuild as mb
import gmso

/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/foyer/forcefield.py:33: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import iter_entry_points, resource_filename
/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/pkg_resources/__init__.py:3144: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/c

In [2]:
# Load structure from recipes (delivered with mBuild), SMILES string, or external pdb/mol2 file
from mbuild.lib.molecules.water import WaterTIP3P
water = WaterTIP3P()
water_box = mb.fill_box(water, box=[5,5,5], n_compounds=100)
water_box.visualize()



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
pnipam_co_aac = mb.load("CC(C(=O)NC(C)C)C(C(=O)O)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)O)(C)C", smiles=True)
pnipam_co_aac.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#Visualizing the 
"""Visualization utilities"""
print(water_box.print_hierarchy(show_tree=False))  # print_hierarchy() in normal colab
#Optionally, you can save your topology to .pdb, .mol2, .xyz, etc.
water_box.save("water_box.mol2", overwrite=True)
water_box.visualize()

In [4]:
#Loading forcefields
warnings.filterwarnings("ignore", category=DeprecationWarning) 

tip3p = gmso.ForceField("/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/foyer/forcefields/xml/tip3p.xml")
oplsaa = gmso.ForceField("OPLSAA")
gaff = gmso.ForceField("/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/foyer/forcefields/xml/gaff.xml")
gaff

/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/gmso/core/atom_type.py:236: UserWarning: Masses are assumed to be in units of g/mol
  warnings.warn(UNIT_WARNING_STRING.format("Masses", "g/mol"))
/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/gmso/core/atom_type.py:248: UserWarning: Charges are assumed to be in units of elementary charge
  warnings.warn(UNIT_WARNING_STRING.format("Charges", "elementary charge"))


<ForceField Forcefield,
 91 AtomTypes,
 928 BondTypes,
 5315 AngleTypes,
 653 DihedralTypes,
 35 ImproperType,
 0 PairPotentialType,
 id: 6083633712>

In [ ]:
#Adding forcefield to pre-loaded molecules or simulation boxes
from gmso.parameterization import apply

#water_top = water_box.to_gmso()
pnipam_top = pnipam_co_aac.to_gmso()

#water_ptop = apply(water_top, tip3p, identify_connections=True)
pnipam_ptop = apply(pnipam_top, gaff, identify_connections=True)

In [ ]:
display(water_ptop.sites[0].atom_type.expression)
print(f"{water_ptop.sites[0].atom_type.expression}")

In [ ]:
display(pnipam_ptop.sites[0].atom_type.expression)
print(f"{pnipam_ptop.sites[0].atom_type.expression}")

In [ ]:
"""Utility to output system as Dataframe"""
water_ptop.to_dataframe(site_attrs=["atom_type.parameters"])
pnipam_ptop.to_dataframe(site_attrs=["atom_type.parameters"])

In [ ]:
water_ptop.save("water_box.lammps", overwrite=True)
!cat water_box.lammps

In [ ]:
pnipam_ptop.save("pnipam.lammps", overwrite=True)
!cat pnipam.lammps

In [8]:
#pnipam = mb.load("CC(C)NC(=O)C=C", smiles=True)
pnipam = mb.load("CC(C)NC(=O)C=C", smiles=True)
pnipam.visualize()
#C=CC(=O)O

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [15]:
pnipam.name = 'PNP'
pnipam.energy_minimize(forcefield='oplsaa', steps=10**4)

FoyerError: Found no types for atom 1 (6).

In [7]:
aac = mb.load("C=CC(=O)O", smiles=True)
aac.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [6]:
#pnipam = mb.load("CC(C)NC(=O)C=C", smiles=True)
pnipamco = mb.load("CC(C)NC(=O)CC(C)(C(=O)O)", smiles=True)
pnipamco.visualize()
#C=CC(=O)O

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#pnipam_co_aac = mb.load("CC(C(=O)NC(C)C)C(C(=O)O)(C)", smiles=True)
pnipam_co_aac = mb.load("CC(C(=O)NC(C)C)C(C(=O)O)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)O)(C)C", smiles=True)
pnipam_co_aac.visualize()